In [32]:
'''
Skeleton code for resolution theorem prover for propositional logic

Yoonsuck Choe
10/20/2021

clause representation: 

  [<positive-proposition-list>, <negative-proposition-list>]

proposition-list representation: 

  [ 'p', 'q', 'r' ]

  Note: 'p', 'q', 'r', 's' are propositions : CASE SENSITIVE!

Example clause: 

  [['p', 'q'] ['r']] : This is clause P v Q v ~R 

'''

DEBUG=True

#---------------------------------------------------------------------
def rm_item(lst, item):  
#---------------------------------------------------------------------
  '''
  function: rm_pred()
  
  remove item from list
  - return [] if empty, not None
  '''
  lst.remove(item)

  if lst == None:
    return []
  else: 
    return lst


#---------------------------------------------------------------------
def mk_unique(clause):
#---------------------------------------------------------------------
  '''
  remove redundant propositions in the clause
  '''
  pos = clause[0]
  neg = clause[1]

  pos = list(set(pos))
  neg = list(set(neg))

  return [pos, neg]

#---------------------------------------------------------------------
def resolve(clause1, clause2):
#---------------------------------------------------------------------
  ''' 
  function: resolve() 

  [['p', 'q'] ['r']] : clause P v Q v ~R 

  resolve clauses c1 and c2 
  - if resolvable returns resolvent
  - if not resolvable returns False
  - returns empty clause [[], []] when Falsified

  '''

  # remove any redundant propositions in the clauses
  c1 = mk_unique(clause1)
  c2 = mk_unique(clause2)

  count = 0

  # print clauses
  if DEBUG:
    print("resolving: "+str(c1)+" and "+str(c2))

  # c1 pos vs. c2 neg
  for p1 in c1[0]:
    for p2 in c2[1]:
      if p1==p2:
        c1 = [rm_item(c1[0],p1), c1[1]]
        c2 = [c2[0], rm_item(c2[1],p2)]
        count = count+1

  # c2 pos vs. c1 neg
  for p1 in c1[1]:
    for p2 in c2[0]:
      if p1==p2:
        c1 = [c1[0], rm_item(c1[1],p1)]
        c2 = [rm_item(c2[0],p2), c2[1]]
        count = count+1

  # check for multiple matches or no match and abort if so.
  if count>1 or count==0:
    return False
  
  # compute resolvent
  c1[0].extend(c2[0])
  c1[1].extend(c2[1])

  # make unique and return
 
  return mk_unique(c1)



#---------------------------------------------------------------------
def add_unique(thm, clause):
#---------------------------------------------------------------------
  ''' 
  add clause to thm if no identical clause exists
  prevents getting locked in a loop adding more clauses

  '''
  pos = set(clause[0])
  neg = set(clause[1])
  POS = set()
  NEG = set()
  for i,c in thm:
    POS.update(set(c[0]))
    NEG.update(set(c[1]))
    # print(f'/// +: {pos, POS}\n/// -: {neg,NEG}')
    if pos==POS and neg==NEG: return -1
    POS.clear()
    NEG.clear()
  thm.append([len(thm),clause])
  thm[:] = thm # [:] so Python updates the global thm
  print("clause "+str(len(thm)-1)+": ",end="")
  print_clause(clause)



#---------------------------------------------------------------------
def null_p(clause):
#---------------------------------------------------------------------
  '''
  function null_p()

  check if "False" is derived 
  - returns True if empty clause
  - returns False if non-empty clause
  '''

  if (type(clause) is list) and len(clause[0]) + len(clause[1]) == 0:
    return True
  else:
    return False



#---------------------------------------------------------------------
def print_clause(clause):
#---------------------------------------------------------------------
  '''
  function print_clause

  See function resolve() for clause representation.

  '''

  s = ""
  for pos in clause[0]:
    s = s+pos+"v"
  
  for neg in clause[1]:
    s = s+"~"+neg+"v"
 
  if len(s)==0:
    print("NULL")
  else:
    n=len(s)
    print(s[0:(n-1)])

#------------------------------------------------------
# test resolve() function
#------------------------------------------------------

print("\ntest1")
test1 = resolve([['p', 'q'], ['r']], [[], ['p']])
print("resolvent = "+str(test1))
print("null? "+str(null_p(test1)))

print("\ntest2")
test2 = resolve([['p'], []], [[], ['p']])
print("resolvent = "+str(test2))
print("null? "+str(null_p(test2)))

print("\ntest3")
test3 = resolve([['p', 'q'], ['r']], [['r'], ['p']])
print("resolvent = "+str(test3))
print("null? "+str(null_p(test3)))

print("\ntest4")
test4 = resolve([['p', 'q'], ['r']], [['r'], []])
print("resolvent = "+str(test4))
print("null? "+str(null_p(test4)))


print("\ntest5")
test5 = resolve([['p', 'q'], ['r']], [['p', 'r'], []])
print("resolvent = "+str(test5))
print("null? "+str(null_p(test5)))


# edge case not being handled correctly: p v q v ~r v ~r  vs. p v r should give p v q, not p v q v ~r 
# - now this is fixed, with the use of mk_unique()
print("\ntest6")
test6 = resolve([['p', 'q'], ['r', 'r']], [['p', 'r'], []])
print("resolvent = "+str(test6))
print("null? "+str(null_p(test6)))



test1
resolving: [['p', 'q'], ['r']] and [[], ['p']]
resolvent = [['q'], ['r']]
null? False

test2
resolving: [['p'], []] and [[], ['p']]
resolvent = [[], []]
null? True

test3
resolving: [['p', 'q'], ['r']] and [['r'], ['p']]
resolvent = False
null? False

test4
resolving: [['p', 'q'], ['r']] and [['r'], []]
resolvent = [['p', 'q'], []]
null? False

test5
resolving: [['p', 'q'], ['r']] and [['p', 'r'], []]
resolvent = [['p', 'q'], []]
null? False

test6
resolving: [['p', 'q'], ['r']] and [['p', 'r'], []]
resolvent = [['p', 'q'], []]
null? False


In [35]:
'''
Using the resolve() function and the null_p function, implement the two-pointer method for propositional logic theorem proving
- use set of support

theorem representation:

[ [<index> <clause1>], [<index> <clause2>], ... ]

index : integer 

clause representation:

[<positive-proposition-list>, <negative-proposition-list>]

proposition-list representation: 

[ 'p', 'q', 'r' ]

'''

# define your theorem, as a set of indexed clauses

theorem = [    # pos-list      # neg-list 
           [0, [['p','q'],    ['r']]],       # clause 0     P v Q v ~R
           [1, [['s'],        ['q']]],       # clause 1     S v ~Q 
           [2, [['t'],        ['s']]],       # clause 2     T v ~S 
           [3, [[],           ['p']]],       # clause 3     ~P   - negated conclusion starts here (goal clause index = 3)
           [4, [['r'],        []]],          # clause 4     R   
           [5, [[],           ['t']]]        # clause 5     ~T  
          ]

# enter the example theorm in the homework
theorem2 = [    # pos-list      # neg-list 
           [0, [['s','q'],    ['p']]],       # clause 0     S v ~P v Q
           [1, [[],           ['r','q']]],   # clause 1     ~R v ~Q 
           [2, [['p','t'],    ['r']]],       # clause 2     P v T v ~R
           [3, [['r'],        []]],          # clause 3     R
           [4, [[],           ['s']]],       # clause 4     ~S
           [5, [[],           ['t']]],       # clause 5     ~T
           ] 

# the other example theorem from the homework (unicorn theorem)
theorem3 = [
           [0, [['i'],        ['m']]],
           [1, [['m'],        ['i']]],
           [2, [['m','l'],    []]],
           [3, [['h'],        ['i']]],
           [4, [['h'],        ['l']]],
           [5, [['g'],        ['h']]],
           [6, [[],           ['h']]],
           ] # ~m and ~g as conclusions do not allow us to solve by resolution


#------------------------------------------------------
# prover() function : you must implement this function 
#---------------------------------------------------------------------
def prover(thm, goal):
#---------------------------------------------------------------------
  '''
  function prover: implement this

  two-pointer method, with set of support
  - arguments: 
    thm : theorem
    goal : integer index (clause number where the negated conclusion starts)
  - show resolution steps
  - if null_p checks, return True (theorem proven)
  - otherwise return False
  '''

  print('\nprover():\n\nTheorem:')

  # Print theorem
  for clause in thm:
    print("clause "+str(clause[0])+": ",end="")
    print_clause(clause[1])
  # Print goal
  print('Goal clause index=',end='')
  print(goal)

  # print('Test resolve: clause 0 and clause 1')
  # print_clause(resolve(theorem[0][1], theorem[1][1]))

  # print('\n\n--- must print the resolution steps\n\n\n')
  i = goal
  j = 0
  while i<len(thm):
    while j<i:
      r = resolve(thm[i][1],thm[j][1])
      if r:
        # thm.append([len(thm),r])
        # print("clause "+str(len(thm)-1)+": ",end="")
        # print_clause(r)
        add_unique(thm,r)
        if null_p(r):
          return 'success'
      j+=1
    i+=1
    j=0

  
  return False

#------------------------------------------------------
# test prover() function
#------------------------------------------------------

# Test prover on theorems

print(prover(theorem, 3))
print(prover(theorem2, 3))
print(prover(theorem3, 6))

# Test clause printer

# print("Testing print_clause()")
# print_clause(theorem[0][1])
# print_clause([['p'], ['q']])
# print_clause([['p','s'], ['q','r']])


prover():

Theorem:
clause 0: pvqv~r
clause 1: sv~q
clause 2: tv~s
clause 3: ~p
clause 4: r
clause 5: ~t
Goal clause index=3
resolving: [[], ['p']] and [['p', 'q'], ['r']]
clause 6: qv~r
resolving: [[], ['p']] and [['s'], ['q']]
resolving: [[], ['p']] and [['t'], ['s']]
resolving: [['r'], []] and [['p', 'q'], ['r']]
clause 7: pvq
resolving: [['r'], []] and [['s'], ['q']]
resolving: [['r'], []] and [['t'], ['s']]
resolving: [['r'], []] and [[], ['p']]
resolving: [[], ['t']] and [['p', 'q'], ['r']]
resolving: [[], ['t']] and [['s'], ['q']]
resolving: [[], ['t']] and [['t'], ['s']]
clause 8: ~s
resolving: [[], ['t']] and [[], ['p']]
resolving: [[], ['t']] and [['r'], []]
resolving: [['q'], ['r']] and [['p', 'q'], ['r']]
resolving: [['q'], ['r']] and [['s'], ['q']]
clause 9: sv~r
resolving: [['q'], ['r']] and [['t'], ['s']]
resolving: [['q'], ['r']] and [[], ['p']]
resolving: [['q'], ['r']] and [['r'], []]
clause 10: q
resolving: [['q'], ['r']] and [[], ['t']]
resolving: [['p', 'q'], []] 

In [34]:
theorem4 = [
           [0, [['i'],        ['m']]],
           [1, [['m'],        ['i']]],
           [2, [['m','l'],    []]],
           [3, [['h'],        ['i']]],
           [4, [['h'],        ['l']]],
           [5, [['g'],        ['h']]],
           [6, [[],           ['m']]],
           ]
print(prover(theorem4, 6)) # TODO check if clause exists before adding new one
# this will get stuck in a loop since ~m proves ~i and l
#                                     ~i proves ~m (known)
#                                      l proves h
#                                      h proves g
# and because no additional information can be proved, we continue adding these clauses
# we can output 'failure' if we compare against out list of clauses preventing duplicates from being added
# which will allow our prover to reach the end

theorem5 = [
           [0, [['i'],        ['m']]],
           [1, [['m'],        ['i']]],
           [2, [['m','l'],    []]],
           [3, [['h'],        ['i']]],
           [4, [['h'],        ['l']]],
           [5, [['g'],        ['h']]],
           [6, [[],           ['g']]],
           ]
print(prover(theorem5, 6))



prover():

Theorem:
clause 0: iv~m
clause 1: mv~i
clause 2: mvl
clause 3: hv~i
clause 4: hv~l
clause 5: gv~h
clause 6: ~m
Goal clause index=6
resolving: [[], ['m']] and [['i'], ['m']]
resolving: [[], ['m']] and [['m'], ['i']]
clause 7: ~i
resolving: [[], ['m']] and [['l', 'm'], []]
clause 8: l
resolving: [[], ['m']] and [['h'], ['i']]
resolving: [[], ['m']] and [['h'], ['l']]
resolving: [[], ['m']] and [['g'], ['h']]
resolving: [[], ['i']] and [['i'], ['m']]
resolving: [[], ['i']] and [['m'], ['i']]
resolving: [[], ['i']] and [['l', 'm'], []]
resolving: [[], ['i']] and [['h'], ['i']]
resolving: [[], ['i']] and [['h'], ['l']]
resolving: [[], ['i']] and [['g'], ['h']]
resolving: [[], ['i']] and [[], ['m']]
resolving: [['l'], []] and [['i'], ['m']]
resolving: [['l'], []] and [['m'], ['i']]
resolving: [['l'], []] and [['l', 'm'], []]
resolving: [['l'], []] and [['h'], ['i']]
resolving: [['l'], []] and [['h'], ['l']]
clause 9: h
resolving: [['l'], []] and [['g'], ['h']]
resolving: [['l'], 